In [1]:
# Adapted from: https://github.com/lazyprogrammer/machine_learning_examples/tree/master/rl
import numpy as np
from gridWorldGame import standard_grid, negative_grid,print_values, print_policy

SMALL_ENOUGH = 1e-3
GAMMA = 0.9
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')
# this grid gives you a reward of -0.1
# to find a shorter path to the goal, use negative grid
grid = negative_grid()
print("rewards:")
print_values(grid.rewards, grid)

rewards:
---------------------------
-0.10|-0.10|-0.10| 1.00|
---------------------------
-0.10| 0.00|-0.10|-1.00|
---------------------------
-0.10|-0.10|-0.10|-0.10|


In [2]:
# state -> action
# choose an action and update randomly 
policy = {}
for s in grid.actions.keys():
  policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)

In [3]:
# initial policy
print("initial policy:")
print_policy(policy, grid)

initial policy:
---------------------------
  R  |  R  |  D  |     |
---------------------------
  L  |     |  U  |     |
---------------------------
  L  |  U  |  R  |  D  |


In [4]:
# initialize V(s) - value function
V = {}
states = grid.all_states()
for s in states:
  # V[s] = 0
  if s in grid.actions:
    V[s] = np.random.random()
  else:
    # terminal state
    V[s] = 0

# initial value for all states in grid
print(V)
print_values(V, grid)

{(0, 1): 0.3558994676171062, (1, 2): 0.4055931780682699, (2, 1): 0.08181838714458345, (0, 2): 0.5932760912743917, (2, 2): 0.7732672580562513, (1, 0): 0.14728375020778894, (1, 3): 0, (0, 0): 0.055923225112072594, (0, 3): 0, (2, 0): 0.34913964535661035, (2, 3): 0.5225367394045576}
---------------------------
 0.06| 0.36| 0.59| 0.00|
---------------------------
 0.15| 0.00| 0.41| 0.00|
---------------------------
 0.35| 0.08| 0.77| 0.52|


In [5]:
iteration=0
# repeat until convergence
# when policy does not change, it will finish
while True:
  iteration+=1
  print("values %d: " % iteration)
  print_values(V, grid)
  print("policy %d: " % iteration)
  print_policy(policy, grid)

  # policy evaluation step
  while True:
    biggest_change = 0
    for s in states:
      old_v = V[s]

      # V(s) only has value if it's not a terminal state
      if s in policy:
        a = policy[s]
        grid.set_state(s)
        r = grid.move(a) #reward
        V[s] = r + GAMMA * V[grid.current_state()]
        biggest_change = max(biggest_change, np.abs(old_v - V[s]))

    if biggest_change < SMALL_ENOUGH:
      break

  # policy improvement step
  is_policy_converged = True
  for s in states:
    if s in policy:
      old_a = policy[s]
      new_a = None
      best_value = float('-inf')
      # loop through all possible actions to find the best current action
      for a in ALL_POSSIBLE_ACTIONS:
        grid.set_state(s)
        r = grid.move(a)
        v = r + GAMMA * V[grid.current_state()]
        if v > best_value:
          best_value = v
          new_a = a
      policy[s] = new_a
      if new_a != old_a:
        is_policy_converged = False

  if is_policy_converged:
    break

values 1: 
---------------------------
 0.06| 0.36| 0.59| 0.00|
---------------------------
 0.15| 0.00| 0.41| 0.00|
---------------------------
 0.35| 0.08| 0.77| 0.52|
policy 1: 
---------------------------
  R  |  R  |  D  |     |
---------------------------
  L  |     |  U  |     |
---------------------------
  L  |  U  |  R  |  D  |
values 2: 
---------------------------
-1.00|-1.00|-1.00| 0.00|
---------------------------
-0.99| 0.00|-1.00| 0.00|
---------------------------
-0.99|-0.99|-0.99|-0.99|
policy 2: 
---------------------------
  D  |  U  |  R  |     |
---------------------------
  D  |     |  D  |     |
---------------------------
  D  |  R  |  D  |  D  |
values 3: 
---------------------------
-0.99|-1.00| 1.00| 0.00|
---------------------------
-0.99| 0.00|-0.99| 0.00|
---------------------------
-0.99|-0.99|-0.99|-0.99|
policy 3: 
---------------------------
  D  |  R  |  R  |     |
---------------------------
  D  |     |  U  |     |
---------------------------
  R  

In [6]:
print("final values:")
print_values(V, grid)
print("final policy:")
print_policy(policy, grid)

final values:
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.46| 0.00| 0.80| 0.00|
---------------------------
 0.31| 0.46| 0.62| 0.46|
final policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |
